In [15]:
from os import listdir, path, getcwd, remove
from collections import defaultdict
from tkinter import *
from tqdm.notebook import tqdm
from time import time
from pickle import load, dump
from datetime import datetime

In [16]:
def metrics(fun):
    def wraper(*args, **kwargs):
        start_time = time()
        result = fun(*args, **kwargs)
        print(time() - start_time)
        return result
    return wraper

In [17]:
class Reader:
    def __init__(self, work_dir):
        self.result = defaultdict(dict) 
        arr_book_fact = set(listdir(work_dir))
        if path.exists("dump_result") and path.exists("arr_book"):
            self.result = load(open("dump_result", "rb"))
            arr_book_base = set(load(open("arr_book", "rb")))
            new_book = arr_book_fact - arr_book_base
            self.get_content(work_dir, new_book)
            
            new_book = arr_book_base - arr_book_fact
            if new_book:
                for book in new_book:
                    for word in self.result: 
                        self.result[word].discard(book)
                abs_path = path.join(getcwd(), "dump_result")
                remove(abs_path)
                abs_path = path.join(getcwd(), "arr_book")
                remove(abs_path)   
        else:
            self.get_content(work_dir, arr_book_fact)
        dump(self.result, open("dump_result", "wb"))
        dump(arr_book_fact, open("arr_book", "wb"))
         
            
    def get_content(self, work_dir, arr_book):
        for book in tqdm(arr_book):
            abs_path = path.join(getcwd(), work_dir, book)
            with open(abs_path, encoding="cp1251") as file:
                try:
                    content = file.read()
                except UnicodeDecodeError:
                    with open(abs_path, encoding="utf-8") as file:
                        content = file.read()
            content = content.replace(",", "")
            content = content.replace(".", "")
            content = content.replace(":", "")
            content = content.lower().split()
            content_set = set(content)          
            for word in content_set:             
                i = content.index(word)
                if i > 2:
                    self.result[word][book] = f"{content[i-3]} {content[i-2]} {content[i-1]} {content[i]}"
                else:
                    self.result[word][book] = f"{content[i]} {content[i+1]} {content[i+2]} {content[i+3]}" 
                    
   
    def find_books_title(self, row):
        out_dict = defaultdict(dict)        
        row = row.lower().split(", ")
        out = self.result[row[0]]
        for word in row[1:]:
            out = set(out) & set(self.result[word]) 
        for book in out:
            for word in row:
                out_dict[book][word] = self.result[word][book]
        return out_dict

In [19]:
class Window:
    def __init__(self, reader):
        self._reader = reader
        self.root = Tk()
        self.entry = Entry(self.root, width=50, font=15)
        self.text = Text(self.root, width=50, height=15, font=15)
        self.button = Button(self.root, text="OK", width=5, height=19, command=self.button_cl)
        self.entry.grid(row=0, column=0)
        self.text.grid(row=1, column=0)
        self.button.grid(row=0, column=1, rowspan=2)

    @metrics
    def button_cl(self):
        row = self.entry.get()
        out = self._reader.find_books_title(row)
        out_text = ""
        for element in out:
            out_text += f"{element} --- \n{out[element]}\n\n"     
        self.text.delete(0.0, 'end')
        self.text.insert(0.0, out_text)

In [20]:
print(datetime.now())
reader = Reader("books")
print(datetime.now())
Window(reader).root.mainloop()

2020-10-14 22:00:19.714706



2020-10-14 22:00:25.111660
0.0
